In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# 全てのデータの読み込み
df_all = pd.read_csv('C:/Users/thisi/OneDrive/デスクトップ/Dateset1.csv')

print("データセットの大きさ:", df_all.shape)
print("\nデータの最初の5行:")
print(df_all.head(500))

# 説明変数と目的変数を分離
X = df_all.drop('CRIM', axis=1)
y = df_all['CRIM']

print("\n説明変数の列:")
print(X.columns)

# パイプラインの定義（正規化とSVR）
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

# ハイパーパラメータの範囲を定義
param_grid = {
    'svr__C': [0.1, 1, 10, 100],
    'svr__epsilon': [0.01, 0.1, 0.5],
    'svr__kernel': ['rbf', 'linear'],
    'svr__gamma': ['scale', 'auto', 0.1, 1]
}

# GridSearchCVの設定と実行
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# 最適なモデルを取得
best_model = grid_search.best_estimator_

print("\n最適なパラメータ:", grid_search.best_params_)
print("最良の負のMSE:", grid_search.best_score_)

# 交差検証を使用してモデルの性能を評価
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')
mse_scores = -cv_scores
r2_scores = cross_val_score(best_model, X, y, cv=5, scoring='r2')

print(f"\n交差検証MSE: {mse_scores.mean():.4f} (+/- {mse_scores.std() * 2:.4f})")
print(f"交差検証R-squared: {r2_scores.mean():.4f} (+/- {r2_scores.std() * 2:.4f})")

# 全データセットでモデルを再学習
best_model.fit(X, y)

# 新しいデータの読み込み（予測したいデータ）
df_new = pd.read_csv('C:/Users/thisi/OneDrive/デスクトップ/Dateset2.csv')

# 新しいデータの説明変数を抽出
X_new = df_new[X.columns]  # 元のデータと同じ列を使用

# 予測の実行
y_pred = best_model.predict(X_new)

# 予測結果をデータフレームに追加
df_new['Predicted_Target'] = y_pred

# 予測結果の表示
print("\n予測結果の最初の5行:")
print(df_new[['Predicted_Target'] + list(X.columns)].head())

# 予測結果をCSVファイルに保存
csv_filename = 'prediction_results2.csv'
df_new.to_csv(csv_filename, index=False)
print(f"\n予測結果を'{csv_filename}'に保存しました。")

データセットの大きさ: (506, 14)

データの最初の5行:
        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
495  0.17899   0.0   9.69     0  0.585  5.670  28.8  2.7986    6  391   
496  0.28960   0.0   9.69     0  0.585  5.390  72.9  2.7986    6  391   
497  0.26838   0.0   9.69     0  0.585  5.794  70.6  2.8927    6  391   
498  0.23912   0.0   9.69     0  0.585  6.019  65.3  2.4091    6  391   
499  0.17783   0.0   9.69     0  0.585  5.569  73.5  2.3999    6  391   

     PTRATIO       B  LSTAT  MEDV  
0       15.3  396.90   4.98  24.0  
1       17.8  396